# **Image generation Using Stable diffusion framework**

This notebook generates images given image-text embeddings. It supports single image and batch image creation. The notebook is structured to align with the objectives outlined in the thesis.

## **1. Environment Setup**
Install necessary libraries and clone the required repository.

In [ ]:
!git clone https://github.com/Reouth/Movie-Character-Identification-With-Perosnalized-Generative-Models.git

%pip install -qq git+https://github.com/huggingface/diffusers.git
%pip install -q accelerate
!pip install bitsandbytes

## **2. Import Libraries**
Load necessary Python libraries and scripts.

In [ ]:
import os
import torch
import gc
import configparser
from torch import autocast
import numpy as np

# Change directory to cloned repository
os.chdir('/content/Movie-Character-Identification-With-Perosnalized-Generative-Models')

from models import Diffusion_generate
from src import data_upload
from src import helper_functions


## **3. Configure Authentication**
Login to Hugging Face to access the Stable Diffusion model.

In [ ]:
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

## **4. Mount Google Drive**
Store and retrieve files from Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **5. Configuration**
Set model parameters and input/output paths.

*   cat_embeds: True for creating character avergage embedding (category embeds) OR False for all embeds in folder
*   Imagic_pipe: True for imagic pipine OR False for SD pipeline

In [ ]:
# Configuration

num_inference_steps = 50  #@param {type:"number"}
resolution = 512  #@param {type:"number"}
height = 512  #@param {type:"number"}
width = 512  #@param {type:"number"}
seed_range = [0]  #@param {type:"raw"}
alpha_range = np.arange(0, 1, 0.05)  #@param {type:"raw"}
guidance_scale_range = list(np.arange(0, 10, 0.5))  #@param {type:"raw"}
cat_embeds = False  #@param {type:"boolean"}
Imagic_pipe = True  #@param {type:"boolean"}
SD_model_name = 'CompVis/stable-diffusion-v1-4'
CLIP_model_name = 'openai/clip-vit-large-patch14'
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
# Path configurations
output_dir = '/content/drive/MyDrive/thesis_OO_SD/Fabelmans_movie/generated_images/mixed_model_embeds'#@param {type:"string"} #output main folder
os.makedirs(output_dir,exist_ok=True)
input_embeddings_path =  '/content/drive/MyDrive/thesis_OO_SD/Fabelmans_movie/Imagic_embeddings/4_ID/1024res/embeds_with_pipe' #@param {type:"string"} #input main folder

text_prompts  =  ["a_red_filtered_photo_of_a_person","a photo with a red filter on it", "a red filtered photo"] #@param {type:"raw"}  # add text prompts to generate here (correlates to embedings text)


# input/output path creation
input_dirs=[]
output_dirs=[]

for text in text_prompts:
  input_dirs.append(os.path.join(imagic_pretrained_path,text.replace(" ","_")))
  output_dirs.append(os.path.join(dir,text.replace(" ","_")))

## **6. Generator**
Run Generateor for multiple image generation for image_text embeddings

In [ ]:
#@title multiple images generator

for input_dir, output_dir in zip(input_dirs,output_dirs):
  os.makedirs(output_dir,exist_ok=True)
  if cat_embeds:
    imagic_cat_embeds = data_upload.upload_cat_embeds(input_dir,CLIP_model_name, device)
    all_embeds = list(imagic_cat_embeds.keys())
  else:
    all_files = set(os.listdir(input_dir))
    imagic_cat_embeds=None
  SD_model.all_generator(all_files,output_dir,input_dir,Imagic_pipe,SD_model_name, CLIP_model_name, device,
                    seed_range,alpha_range,guidance_scale_range,imagic_cat_embeds)

